In [ ]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import math

# torch.index_select
- 选择某一维度的tensor
- 选择某一维度（切片）的tensor

In [ ]:
a = [i for i in range(100)]

In [ ]:
a = np.asarray(a)

In [ ]:
tensor = torch.from_numpy(a).view(5, 2, 10)
tensor

In [ ]:
index = torch.tensor([0])

In [ ]:
print(index)

In [ ]:
selected_tensor = torch.index_select(tensor, -2, index)

In [ ]:
print(selected_tensor)

In [ ]:
x = torch.rand(3, 4)

In [ ]:
x

In [ ]:
indices = torch.tensor([0, 2])

In [ ]:
torch.index_select(x, 0, indices)

# nn.Softmax
- 指定某一维度进行softmax

In [ ]:
t = torch.rand(2, 3, 4)
t

In [ ]:
softmax_layer = nn.Softmax(dim=2)
softmax_layer(t)

# 生成mask tensor
- 生成下三角矩阵

In [ ]:
mask_tensor = torch.ones(0)
batch_size = 2
len_sentence = 10
mask = torch.ones(len_sentence)
for j in range(batch_size):
    for i in range(1, len_sentence + 1):
        masked = mask.clone()
        masked[i:] = 0
        mask_tensor = torch.cat((mask_tensor, masked))
mask_tensor = mask_tensor.contiguous().view(batch_size, len_sentence, -1)
mask_tensor.size()

In [ ]:
print(mask_tensor)

In [ ]:
indices

In [ ]:
for idx in range(len_sentence):
    indices = torch.tensor([idx])
    print(torch.index_select(mask_tensor, 1, indices).squeeze().size())

# CrossEntropyLoss与LogSoftmax+NLLLoss

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
output = loss(input, target)
print(output)
output.backward()

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
print(input)
print(target)
print(m(input))
output = loss(m(input), target)
print(output)
output.backward()

# argmax

In [ ]:
a = torch.rand(3, 4)
a

In [ ]:
print(torch.max(a, 0))

In [ ]:
print(torch.max(a, 1))

In [ ]:
value, index = torch.max(a, 1)
print(index)

In [ ]:
test = torch.rand(3, 4, 5)
test

In [ ]:
test_max = torch.max(test, 2)[1]
test_max

# torch.cat
- 拼接

In [ ]:
b = torch.rand(3, 1)
b

In [ ]:
a = torch.cat([a, b], dim=1)
a

# 测试mask tensor

In [ ]:
BATCH_SIZE = 3
TGT_LENGTH = 5

In [ ]:
def create_mask_tensor():
    mask_tensor = torch.ones(0)
    batch_size = BATCH_SIZE
    len_sentence = TGT_LENGTH
    mask = torch.ones(len_sentence)
    for _ in range(batch_size):
        for i in range(1, len_sentence + 1):
            masked = mask.clone()
            masked[i:] = 0
            mask_tensor = torch.cat((mask_tensor, masked))
    mask_tensor = mask_tensor.contiguous().view(batch_size, len_sentence, -1)
    return mask_tensor.long()

In [ ]:
mask_tensor = create_mask_tensor()

In [ ]:
decoder_input = torch.rand(BATCH_SIZE, TGT_LENGTH)
decoder_input

In [ ]:
decoder_output = torch.rand(BATCH_SIZE * 1).contiguous().view(-1, 1)
decoder_output

In [ ]:
decoder_input[:, 1:2] = decoder_output
decoder_input

In [ ]:
decoder_input[:, 0:3]

In [ ]:
for i in range(TGT_LENGTH):
    mask_cur = mask_tensor[:, i].squeeze().float()
    print(torch.mul(decoder_input, mask_cur))

# 测试SOS符号

In [ ]:
decoder_input = torch.tensor(
    [0 for _ in range(TGT_LENGTH * BATCH_SIZE)]).contiguous().view(
        BATCH_SIZE, TGT_LENGTH)
decoder_input[:, 0] = torch.tensor([3 for _ in range(BATCH_SIZE)])
print(decoder_input)

# 测试mask convolution(casual convolution)

In [ ]:
class MaskConv1d(nn.Conv1d):
    """Masked Conv Kernel for Temporal Convolution"""

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding,
                 dilation):
        super().__init__(in_channels, out_channels, kernel_size, stride,
                         padding, dilation)
        self.mask = torch.tensor([
            0 for _ in range(self.out_channels * self.in_channels *
                             self.kernel_size[0])
        ])
        for idx in range(self.out_channels * self.in_channels *
                         self.kernel_size[0])[::self.kernel_size[0]]:
            for jdx in range(1 + self.kernel_size[0] // 2):
                self.mask[idx + jdx] = 1
        self.mask = self.mask.contiguous().view(
            self.out_channels, self.in_channels, self.kernel_size[0]).float()

    def forward(self, x):
        return F.conv1d(
            input=x,
            weight=self.weight * self.mask,
            stride=self.stride,
            padding=self.padding,
            dilation=self.dilation)

    def show_mask(self):
        return self.mask

    def show_kernel(self):
        return self.weight

    def show_masked_kernel(self):
        return self.mask * self.weight

In [ ]:
def MaskedWeightNormalizedConv1d(in_channels, out_channels, kernel_size,
                                 stride, padding, dropout, dilation):
    """Weight-normalized Conv1d layer"""
    m = MaskConv1d(in_channels, out_channels, kernel_size, stride, padding,
                   dilation)
    std = math.sqrt((4 * (1.0 - dropout)) / (m.kernel_size[0] * in_channels))
    nn.init.normal_(m.weight, mean=0, std=std)
    nn.init.constant_(m.bias, 0)
    return nn.utils.weight_norm(m, dim=2)

In [ ]:
in_channels = 1
out_channels = 1
kernel_size = 5
stride = 1
padding = 2
dropout = 0
dilation = 1
length = 5
batch = 2

In [ ]:
test_conv = MaskedWeightNormalizedConv1d(
    in_channels, out_channels, kernel_size, stride, padding, dropout, dilation)

In [ ]:
test_input = torch.rand(batch, in_channels, length)
test_input

In [ ]:
test_output = test_conv(test_input)
test_output

In [ ]:
test_output[:, :, 2].view(batch, out_channels, 1)

# 可以更改conv的pad，不影响参数

In [ ]:
print(type(test_conv.padding))
print(test_conv.padding)
test_conv.padding = (0, )
print(test_conv.padding)

In [ ]:
test_output = test_conv(test_input)
print(test_output)
print(test_output.size())

In [ ]:
test_conv.show_mask()

In [ ]:
test_conv.show_kernel()

In [ ]:
test_conv.show_masked_kernel()

# 手动padding

In [ ]:
tobepad = torch.rand(batch_size, length)
tobepad

In [ ]:
pad = torch.zeros(batch_size, padding)
pad

In [ ]:
padded = torch.cat([pad, tobepad, pad], dim=1)
padded

# 截取kernel_size window

In [ ]:
for i in range(length):
    print(padded[:, i:i + kernel_size])

# 对输入遮罩

In [ ]:
BATCH_SIZE = 2
TGT_LENGTH = 10

In [ ]:
def create_mask_tensor():
    mask_tensor = torch.ones(0)
    batch_size = BATCH_SIZE
    len_sentence = TGT_LENGTH
    mask = torch.ones(len_sentence)
    for _ in range(batch_size):
        for i in range(1, len_sentence + 1):
            masked = mask.clone()
            masked[i:] = 0
            mask_tensor = torch.cat((mask_tensor, masked))
    mask_tensor = mask_tensor.contiguous().view(batch_size, len_sentence, -1)
    return mask_tensor.float()

In [ ]:
tensor = create_mask_tensor()
tensor

In [ ]:
decoder_input = torch.rand(BATCH_SIZE, TGT_LENGTH)
decoder_input

In [ ]:
for i in range(TGT_LENGTH):
    print(torch.mul(decoder_input, tensor[:, i, :]))

# 制造decoder input

In [ ]:
decoder_input = torch.zeros(BATCH_SIZE, TGT_LENGTH)
decoder_input

In [ ]:
STR_tensor = torch.Tensor([3 for _ in range(BATCH_SIZE)]).view(BATCH_SIZE, -1)
STR_tensor

In [ ]:
decoder_input[:, 0:1] = STR_tensor
decoder_input

In [ ]:
a = np.ones((5, 4))
b = np.ones((5, 4))
a[2][1] = 5
a

In [ ]:
b

In [ ]:
a - b

In [ ]:
np.sum(a - b == 0)

# 测试位置向量

In [ ]:
batch_size = 5
src_len = 10

src = torch.rand([batch_size, src_len])
src

In [ ]:
pos = torch.arange(0, src.shape[1]).unsqueeze(0).repeat(src.shape[0], 1)
pos

# Teacher Forcing Mask Matrix|

In [ ]:
ratio = 0.7

In [ ]:
m = torch.tensor(
    [1 if np.random.uniform() < ratio else 0 for _ in range(3 * 7)]).view(
        3, 7)

m

In [ ]:
1 - m

In [ ]:
m.view(-1)

# CosineLoss

In [ ]:
# loss = torch.nn.CosineEmbeddingLoss()
loss = torch.nn.L1Loss()

In [ ]:
s1 = torch.rand(2, 3, 4)
s2 = torch.rand(2, 3, 4)

In [ ]:
s1

In [ ]:
s2 = torch.rand(2, 3, 4)
# s2[1,:,:] = s1[1,:,:]
s2

In [ ]:
target = torch.tensor([1.0])

In [ ]:
print(loss(s1, s2))

# 测试交叉熵损失

In [ ]:
loss = torch.nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
score = torch.rand(3, 4)
score

In [ ]:
target = torch.tensor([0, 1, 2])
target

In [ ]:
print(loss(score, target))

In [ ]:
print(F.softmax(score, dim=1))

# 测试单词重复度

In [ ]:
train_predicted = torch.tensor([1, 4, 2, 6, 5, 1, 7, 9])
train_predicted

In [ ]:
target = torch.tensor([5, 99, 124, 5123, 7, 5123, 5123, 6])
target

In [ ]:
target = target.numpy()
target

In [ ]:
target_set = set(target)
target_set

In [ ]:
for item in train_predicted.numpy():
    if item in target_set:
        print(item)

In [ ]:
import numpy as np

In [ ]:
a = np.random.rand(3, 5)
a

In [ ]:
b = a.reshape(-1)
b

# fairseq 创建mask
- B:2
- T:10
- C:7

In [ ]:
src_token = torch.tensor(
    [3, 4, 6, 2, 5, 9, 7, 1, 1, 1, 5, 6, 9, 10, 3, 1, 1, 1, 1, 1]).view(2, -1)
src_token

In [ ]:
padding_idx = 1

In [ ]:
encoder_padding_mask = src_token.eq(padding_idx)
encoder_padding_mask

In [ ]:
x = torch.rand(2, 10, 7)
x

In [ ]:
encoder_padding_mask.unsqueeze(-1)

In [ ]:
print(x.size())
print(encoder_padding_mask.size())
x = x.masked_fill(encoder_padding_mask.unsqueeze(-1), 0)
x

# fairseq生成位置向量

In [ ]:
src_token = torch.tensor(
    [3, 4, 6, 2, 5, 9, 7, 0, 0, 0, 5, 6, 9, 10, 3, 0, 0, 0, 0, 0]).view(2, -1).long()
src_token

In [ ]:
mask = src_token.ne(0).long()
mask

In [ ]:
torch.cumsum(mask, dim=1) * mask + 0

In [ ]:
B = 8
Tg = 10
decoder_input = torch.zeros(B,Tg)
decoder_input

In [ ]:
decoder_input[:,0] = 3
decoder_input